In [1]:
import sqlite3
from bs4 import BeautifulSoup
from urllib.request import urlopen
import re

In [17]:
!pip list

Package                Version
---------------------- ---------
beautifulsoup4         4.11.1
brotlipy               0.7.0
certifi                2021.10.8
cffi                   1.14.6
charset-normalizer     2.0.12
colorama               0.4.4
conda                  4.12.0
conda-package-handling 1.8.0
cryptography           36.0.2
greenlet               0.4.13
hpy                    0.0.3
idna                   3.3
libmambapy             0.23.3
mamba                  0.23.3
pip                    22.0.4
pycosat                0.6.3
pyOpenSSL              22.0.0
PySocks                1.7.1
readline               6.2.4.1
requests               2.27.1
ruamel-yaml-conda      0.15.80
setuptools             60.10.0
six                    1.16.0
soupsieve              2.3.1
tqdm                   4.63.0
urllib3                1.26.9
wheel                  0.37.1


In [2]:
url='https://www.slu.se/en/education/programmes-courses/' #webpage of SLU with programmes
page = urlopen(url)
html = page.read().decode("utf-8")
soup = BeautifulSoup(html, "html.parser")
article=soup.find_all('article')
pattern=re.compile(r'<span class="content">.*</span>')
content=pattern.findall(str(article))
pattern=re.compile(r'<span class="headline">.*</span>')
headlines=pattern.findall(str(article))
pattern=re.compile(r'<a href=".*>')
ahref=pattern.findall(str(article))
pattern2=re.compile(r'>.*<')
pattern_ahref=re.compile(r'\/.*\/')
SLU_programmes=[]
for i in range(23):
    programme = pattern2.findall(headlines[i])
    content_i = pattern2.findall(content[i])
    ahref_i=pattern_ahref.findall(ahref[i])[0]
    ahref_i=ahref_i[1:-1].split('/')
    if ahref_i[3] == 'bachelors-programmes':
        type_programme='Bachelor'
    else:
        type_programme='Master'

    SLU_programmes.append([programme[0][1:-1],ahref_i[0],type_programme,120,content_i[0][1:-1]])    

In [4]:
connection = sqlite3.connect('SLU_english_programmes.db')

In [5]:
query = '''CREATE TABLE SLU_programmes(
                            id INTREGER PRIMARY KEY,
                            name TEXT,
                            language TEXT,
                            level TEXT,
                            length INTREGER,
                            question TEXT)
                            '''

In [6]:
connection.execute(query)

In [7]:
insertion_query = '''INSERT INTO 
                     SLU_programmes (name,language, level, length, question)
                     VALUES (?, ?, ?, ?, ?)
                     '''

In [8]:
connection.executemany(insertion_query, SLU_programmes)
connection.commit()

In [9]:
connection.close()